In [42]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib, json
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/BrunoMattos/Documents2/Dev/stocklab_data/Firestore')
from uploadToFirestore import uploadDocumentToFirestore
import os
#print(os.listdir("./dre_input"))
itr_input = './input_cvm/itr/'
dfp_input = './input_cvm/dfp/'
# print(os.listdir("./input_cvm/itr"))

In [164]:
MapaNiveis = {
    'Ativo Total' : '1',
        'Ativo Circulante': '1.1',
            'Caixa e Equivalentes de Caixa': '1.1.1',
            'Aplicações Financeiras': '1.1.2',
            'Contas a Receber': '1.1.3',
            'Estoques': '1.1.4',
        'Ativo Não Circulante': '1.2',
            'Ativo Realizável a Longo Prazo': '1.2.1',
            'Investimentos' : '1.2.2',
            'Imobilizado' : '1.2.3',
            'Intangível' : '1.2.4',

    'Passivo Total': '2',
        'Passivo Circulante' : '2.1',
            'Empréstimos e Financiamentos': '2.1.1',
        'Passivo Não Circulante' : '2.2',
            'Empréstimos e Financiamentos a Longo Prazo': '2.2.1',
        'Patrimônio Líquido' : '2.3',
                'Capital Social Realizado': '2.3.1',
                'Reservas de Capital': '2.3.2',
                'Reservas de Lucros': '2.3.3',
                'Lucros/Prejuízos Acumulados': '2.3.4',
}
MapNivel1 = {
    'Ativo Total' : 'Ativo Total',
    'Passivo Total' : 'Passivo Total',
}
MapNivel2 = {
    #Ativos
    'Ativo Circulante' : 'Ativo Circulante', 
    'Ativo Não Circulante': 'Ativo Não Circulante',
    'Ativo Realizável a Longo Prazo' : 'Ativo Não Circulante',
    'Ativo Permanente': 'Ativo Não Circulante',
    #Passivos
    'Passivo Circulante' : 'Passivo Circulante',
    'Passivo Não Circulante' : 'Passivo Não Circulante',
    'Patrimônio Líquido' : 'Patrimônio Líquido',
    'Patrimônio Líquido Consolidado' : 'Patrimônio Líquido',
}
MapNivel3 = {
    #Ativo Circulante - subitens
    'Caixa e Equivalentes de Caixa':'Caixa e Equivalentes de Caixa',
    'Disponibilidades':'Caixa e Equivalentes de Caixa',
    'Aplicações Financeiras':'Aplicações Financeiras',
    'Aplicações Interfinanceiras de Liquidez':'Aplicações Financeiras',
    'Contas a Receber':'Contas a Receber',
    'Estoques': 'Estoques',
    #Ativo Não Circulante - subitens
    'Ativo Realizável a Longo Prazo':'Ativo Realizável a Longo Prazo',
    'Investimentos':'Investimentos',
    'Imobilizado': 'Imobilizado',
    'Intangível': 'Intangível',
    #Passivo Circulante - subitens
    'Empréstimos e Financiamentos':'Empréstimos e Financiamentos',
    #Passivo Não Circulante - subitens
    'Empréstimos e Financiamentos a Longo Prazo': 'Empréstimos e Financiamentos a Longo Prazo',
    #Patrimonio Liquido - subitens
    'Capital Social Realizado': 'Capital Social Realizado',
    'Reservas de Capital': 'Reservas de Capital',
    'Reservas de Lucros': 'Reservas de Lucros',
    'Lucros/Prejuízos Acumulados': 'Lucros/Prejuízos Acumulados',
}
MapGrupo = {
    'DF Consolidado - Balanço Patrimonial Ativo': 'Ativo Consolidado',
    'DF Individual - Balanço Patrimonial Ativo': 'Ativo Individual',
    'DF Consolidado - Balanço Patrimonial Passivo': 'Passivo Consolidado',
    'DF Individual - Balanço Patrimonial Passivo': 'Passivo Individual',
}

In [162]:
#BALANÇO PATRIMONIAL ATIVO

BPA_ITR_CON = pd.DataFrame() 
path = itr_input + 'itr_cia_aberta_bpa_con_'
for year in range(2015,2020,1):
    BPA_ITR_CON = pd.concat([BPA_ITR_CON, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

BPA_ITR_IND = pd.DataFrame()
path = itr_input + 'itr_cia_aberta_bpa_ind_'
for year in range(2015,2020,1):
    BPA_ITR_IND = pd.concat([BPA_ITR_IND, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

BPA = pd.concat([BPA_ITR_CON, BPA_ITR_IND]) 

#BALANÇO PATRIMONIAL PASSIVO
BPP_ITR_CON = pd.DataFrame() 
path = itr_input + 'itr_cia_aberta_bpp_con_'
for year in range(2015,2020,1):
    BPP_ITR_CON = pd.concat([BPP_ITR_CON, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

BPP_ITR_IND = pd.DataFrame()
path = itr_input + 'itr_cia_aberta_bpp_ind_'
for year in range(2015,2020,1):
    BPP_ITR_IND = pd.concat([BPP_ITR_IND, pd.read_csv(path+str(year)+'.csv', thousands=',', sep=';', encoding='latin-1')])

BPP = pd.concat([BPP_ITR_CON, BPP_ITR_IND]) 

BP = pd.concat([BPA,BPP])

In [163]:
#Drop ORDEM_EXERC==PENÚLTIMO
BP=BP[BP.ORDEM_EXERC=='ÚLTIMO']
print('1 - Gerando NIVEL ...')
def getLevel(row):
    return len(row['CD_CONTA'].split('.'))
BP['NIVEL']=BP.apply(lambda r: getLevel(r), axis=1)

1 - Gerando NIVEL ...


In [165]:

#Filtrar para somente NIVEL 1
print('2 - Filtrar para somente NIVEL 1')
BP_NIVEL1 = BP[BP.NIVEL==1]
BP_NIVEL1['DESC_SIMPLES']=BP_NIVEL1['DS_CONTA'].map(MapNivel1)
BP_NIVEL1.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL1['CD_CONTA']=BP_NIVEL1['DESC_SIMPLES'].map(MapaNiveis)
#Filtrar para somente NIVEL 2
print('3 - Filtrar para somente NIVEL 2')
BP_NIVEL2 = BP[BP.NIVEL==2]
BP_NIVEL2['DESC_SIMPLES']=BP_NIVEL2['DS_CONTA'].map(MapNivel2)
BP_NIVEL2.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL2['CD_CONTA']=BP_NIVEL2['DESC_SIMPLES'].map(MapaNiveis)
#Filtrar para somente NIVEL 3
print('4 - Filtrar para somente NIVEL 3')
BP_NIVEL3 = BP[BP.NIVEL==3]
#Alterando 'Empréstimos e Financiamentos do Passivo Não Circulante para diferenciar do Passivo Circulante
mask = ((BP_NIVEL3.DS_CONTA=='Empréstimos e Financiamentos') & (BP_NIVEL3.CD_CONTA=='2.02.01'))
BP_NIVEL3.loc[mask,'DS_CONTA'] = 'Empréstimos e Financiamentos a Longo Prazo'
BP_NIVEL3['DESC_SIMPLES']=BP_NIVEL3['DS_CONTA'].map(MapNivel3)
BP_NIVEL3.dropna(subset=['DESC_SIMPLES'], inplace=True)
BP_NIVEL3['CD_CONTA']=BP_NIVEL3['DESC_SIMPLES'].map(MapaNiveis)
#Concat Nivel 1,2 e 3
print('5 - Concatenando todos níveis')
BP=pd.concat([BP_NIVEL1,BP_NIVEL2,BP_NIVEL3])
#Map Grupo (Individual, Consolidado)
print('6 - Finalizando')
BP['GRUPO_DFP']=BP['GRUPO_DFP'].map(MapGrupo)
#Mapeamento da Escala dos Valores
BP['ESCALA'] = BP.ESCALA_MOEDA.map({'MIL': 1000, 'MILHAR': 1000, 'UNIDADE': 1})
#FIM
BP.DT_FIM_EXERC=pd.to_datetime(BP.DT_FIM_EXERC)
#Drop colunas desnecessárias
BP.drop(['DT_REFER', 'VERSAO', 'ESCALA_MOEDA', 'MOEDA', 'ORDEM_EXERC', 'DS_CONTA'], axis=1, inplace=True)
    

2 - Filtrar para somente NIVEL 1
3 - Filtrar para somente NIVEL 2
4 - Filtrar para somente NIVEL 3
5 - Concatenando todos níveis
6 - Finalizando


In [166]:
def getTrim(month):
    if(month==2 or month==3):
        return 1
    elif(month==5 or month==6):
        return 2
    elif(month==8 or month==9):
        return 3
    elif(month==11 or month==12):
        return 4
    else:
        return np.nan

BP['TRIM'] = BP.DT_FIM_EXERC.map(lambda d: getTrim(d.month))
BP.dropna(inplace=True)
BP['YEAR'] = BP.DT_FIM_EXERC.dt.year

In [167]:
def calculateAV(r):
    yr = r.YEAR
    trim = r.TRIM
    valor = r.VL_CONTA
    ativo_total = df[(df.YEAR==yr) & (df.TRIM == trim) & (df.CD_CONTA == '1')]
    passivo_total = df[(df.YEAR==yr) & (df.TRIM == trim) & (df.CD_CONTA == '2')]
    if(len(ativo_total)):
        baseValue = ativo_total['VL_CONTA'].values[0]
    elif(len(passivo_total)):
        baseValue = passivo_total['VL_CONTA'].values[0]
    else:
        baseValue=0
    if baseValue!=0:
        return valor/baseValue
    else:
        return np.nan


In [168]:
# cvm = 94
# cvm=1155
# cvm=24228

numberOfCVM = BP.CD_CVM.nunique()
count=0

for cvm in BP.CD_CVM.unique()[0:1]:
    cvm=4170
    count=count+1
    print(str(count) + '/' + str(numberOfCVM) + ' - cvm: ' + str(cvm))
    DF = BP[(BP.CD_CVM==cvm)]
    for grupo in ['Ativo Consolidado', 'Passivo Consolidado','Ativo Individual', 'Passivo Individual']:
        print('Grupo: ' + grupo)
        df = DF[DF.GRUPO_DFP == grupo]
        if(len(df)):
            cnpj =  df.iloc[0]['CNPJ_CIA']  
            denom_cia =  df.iloc[0]['DENOM_CIA']
            df = df.sort_values(['DESC_SIMPLES', 'DT_FIM_EXERC'])
            df.reset_index(inplace=True, drop=True)
            
            #Drop columns
            df.drop(['DT_FIM_EXERC', 'CNPJ_CIA'], axis=1, inplace=True)
            
            #Sort para garantir o sequenciamento correto
            df = df.sort_values(['DESC_SIMPLES', 'YEAR','TRIM']).reset_index(drop=True)
            #Calculate Variação entre Trimestre - AH (Análise Horizontal)
            cutTwo = (df.DESC_SIMPLES == df.DESC_SIMPLES.shift(1)) 
            AH = []
            for i, row in df.iterrows():
                if cutTwo[i]:
                    if df.iloc[i-1]['VL_CONTA']!=0:
                        ah = (df.iloc[i]['VL_CONTA']-df.iloc[i-1]['VL_CONTA'])/df.iloc[i-1]['VL_CONTA']
                    else:
                        ah = np.nan
                    AH.append(ah)
                else:
                    AH.append(np.nan)
            df['AH']=pd.Series(AH)

            #Sort novamente
            df = df.sort_values(['YEAR','TRIM','CD_CONTA']).reset_index(drop=True)
            #Cálculo da Variação Vertical no Trimestre - AV (Análise Vertical)
            df['AV'] = df.apply(lambda r: calculateAV(r), axis=1)

            #Adicionando o TRIM_ANO como coluna
            df['TRIM_ANO']=df.apply(lambda r: "%dT%d" %(r['TRIM'],r['YEAR']), axis=1)

            for ano in df.YEAR.unique()[0:1]:
                ano=2019
                #Gravando arquivo
                if(len(df[df.YEAR==ano])):
                    print(df[df.YEAR==ano][['TRIM_ANO','CD_CONTA','DESC_SIMPLES', 'VL_CONTA', 'AH', 'AV']].head(50))
                    data = df[df.YEAR==ano][['TRIM_ANO','CD_CONTA','DESC_SIMPLES', 'VL_CONTA', 'ESCALA', 'TRIM', 'YEAR', 'NIVEL', 'AH', 'AV']].to_dict('records')
                else:
                    data = []
                
                data = {
                    "cvm": int(cvm),
                    "ano": int(ano),
                    "cnpj": cnpj,
                    "denom_cia": denom_cia,
                    "grupo": grupo,
                    "data" : data,
                }
                path = './output_cvm/bp/'+str(cvm)+'/'+grupo+'/'
                if not os.path.exists(path):
                    os.makedirs(path)
                with open(path+str(ano)+'.json', 'w') as outfile:
                    json.dump(data, outfile, indent=2)
                    print('  '+ str(cvm) +  ' - ' + grupo +' - ' + str(ano)+'.json gravado - ' + str(df[df.YEAR==ano].size))
        else:
            print('  ' + str(cvm) +  ' - ' + grupo +' - ' + 'EMPTY')


1/751 - cvm: 4170
Grupo: Ativo Consolidado
    TRIM_ANO CD_CONTA                    DESC_SIMPLES     VL_CONTA        AH  \
132   1T2019        1                     Ativo Total  366875555.0  0.071863   
133   1T2019      1.1                Ativo Circulante   58742235.0 -0.030430   
134   1T2019    1.1.1   Caixa e Equivalentes de Caixa   19512943.0 -0.201087   
135   1T2019    1.1.2          Aplicações Financeiras          0.0       NaN   
136   1T2019    1.1.3                Contas a Receber   10729877.0 -0.037686   
137   1T2019    1.1.4                        Estoques   19129779.0  0.178053   
138   1T2019      1.2            Ativo Não Circulante  308133320.0  0.093863   
139   1T2019    1.2.1  Ativo Realizável a Longo Prazo   67650006.0  0.409740   
140   1T2019    1.2.2                   Investimentos   12750666.0  0.012109   
141   1T2019    1.2.3                     Imobilizado  194454009.0  0.023891   
142   1T2019    1.2.4                      Intangível   33278639.0  0.066954 